In [1]:
using DataFrames, CSV
#using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [2]:
races = ["Sakhir", "Jeddah", "Melbourne", "Shanghai", "Baku", "Miami", "Imola", "Monaco", "Barcelona", "Montreal", "Spielberg", "Silverstone", "Budapest", "Spa", "Zandvoort", "Monza", "Singapore", "Suzuka", "Lusail", "Austin", "Mexico City", "Sao Paulo", "Las Vegas","Yas Marina"]
races_circuitref = ["bahrain", "jeddah", "albert_park", "shanghai", "baku", "miami", "imola", "monaco", "catalunya", "villeneuve", "red_bull_ring", "silverstone", "hungaroring", "spa", "zandvoort", "monza", "marina_bay", "suzuka", "losail", "americas", "rodriguez", "interlagos", "las_vegas", "yas_marina"];

In [3]:
#import Data
all_df = CSV.read("circuits.csv", DataFrame)
first(all_df,5)

,circuitId,circuitRef,name,location,country
,Int64,String15,String,String31,String15
1,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia
2,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia
3,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain
4,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain
5,5,istanbul,Istanbul Park,Istanbul,Turkey


In [4]:
select!(all_df, "circuitRef", "location", "lat", "lng")

#only keep rows for the races that we care about, i.e. 2024 schedule
all_df[in(races_circuitref).(all_df.circuitRef), :]

,circuitRef,location,lat,lng
,String15,String31,Float64,Float64
1,albert_park,Melbourne,-37.8497,144.968
2,bahrain,Sakhir,26.0325,50.5106
3,catalunya,Montmeló,41.57,2.26111
4,monaco,Monte-Carlo,43.7347,7.42056
5,villeneuve,Montreal,45.5,-73.5228
6,silverstone,Silverstone,52.0786,-1.01694
7,hungaroring,Budapest,47.5789,19.2486
8,spa,Spa,50.4372,5.97139
9,monza,Monza,45.6156,9.28111


In [5]:
using Geodesy

# Gives distance between two circuits in km
function dist(c1, c2)
    circuit1 = all_df[c1, :]
    circuit2 = all_df[c2, :]
    lat1 = circuit1[:lat]
    lng1 = circuit1[:lng]
    lat2 = circuit2[:lat]
    lng2 = circuit2[:lng]
    return euclidean_distance(LLA(lat1, lng1, 0), LLA(lat2, lng2, 0)) / 1000
end

dist(1,12)

12386.36377883742

In [6]:
#create a matrix of distances between each circuit
distance_matrix=[dist(i,j) for i in 1:24, j in 1:24]

24×24 Matrix{Float64}:
     0.0    6067.92  10361.5    12339.2    …   7556.33   10100.7    10082.2
  6067.92      0.0    5818.24    9417.92       4922.78    5418.79   12100.1
 10361.5    5818.24      0.0     4611.4        7542.94     447.152  10994.7
 12339.2    9417.92   4611.4        0.0        9252.61    5017.04    9327.96
 11597.7    7749.32   2536.63    2254.5        8118.92    2941.47   10454.3
 12232.2    9109.4    4255.09     486.382  …   8937.29    4658.66    9644.26
 12317.0   11632.9    9204.84    5699.13       9431.36    9461.91    8271.26
 12320.5    9310.67   4630.56     592.452      8888.76    5021.37    9631.6
 12365.8    9443.78   5024.18    1192.61       8671.2     5392.55    9750.33
 12206.6    9011.25   4339.68     991.471      8548.98    4721.25    9940.26
     ⋮                                     ⋱                        
  7584.88   5116.5    7690.07    9296.91   …    225.276   7480.79   12643.6
  7518.32   3711.89   6497.96    8867.44       1477.14    6227.66  

In [7]:
function powerset(x::AbstractSet)   
    result = Set([Set()])
    for  elem in x, j in result
        union!(result, Set([union(j,elem)]))
    end
    delete!(result, Set())
    result
end;

In [8]:
"""
Returns a `DataFrame` with the values of the variables from the JuMP container `var`.
The column names of the `DataFrame` can be specified for the indexing columns in `dim_names`,
and the name of the data value column by a Symbol `value_col` e.g. :Value
"""
function convert_jump_container_to_df(var::JuMP.Containers.DenseAxisArray;
    dim_names::Vector{Symbol}=Vector{Symbol}(),
    value_col::Symbol=:Value)

    if isempty(var)
        return DataFrame()
    end

    if length(dim_names) == 0
        dim_names = [Symbol("dim$i") for i in 1:length(var.axes)]
    end

    if length(dim_names) != length(var.axes)
        throw(ArgumentError("Length of given name list does not fit the number of variable dimensions"))
    end

    tup_dim = (dim_names...,)

    # With a product over all axis sets of size M, form an Mx1 Array of all indices to the JuMP container `var`
    ind = reshape([collect(k[i] for i in 1:length(dim_names)) for k in Base.Iterators.product(var.axes...)],:,1)

    var_val  = value.(var)

    df = DataFrame([merge(NamedTuple{tup_dim}(ind[i]), NamedTuple{(value_col,)}(var_val[(ind[i]...,)...])) for i in 1:length(ind)])

    return df
end;

In [9]:
races = 10
home = 1
TH = 1
DH = 2

2

In [10]:
V = Set(1:races)
V_0 = setdiff(V, [home])
all_S = powerset(V_0);

In [11]:
for S in all_S
    if length(S) < 2
        delete!(all_S, S)
    end
    if length(S) > length(V) - 2
        delete!(all_S, S)
    end
    if S == V_0
        delete!(all_S, S)
    end
end

In [13]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1)
set_optimizer_attribute(model, "Threads", 20)

# variables
@variable(model, x[i in V, j in V], Bin)
@variable(model, subloops >= 1)
@variable(model, double_header[i in V_0, j in V_0], Bin)
@variable(model, triple_header[i in V_0, j in V_0, q in V_0], Bin)

# constraints
# each circuit can only be visited once
@constraint(model, only_one_in[j in V_0], sum(x[i, j] for i in V) == 1)
@constraint(model, only_one_out[i in V_0], sum(x[i, j] for j in V) == 1) 

# we cannot go from a circuit to itself
@constraint(model, no_self_connect[i in V], x[i, i] == 0)

# we must start and end at home and the number of loops away from home is subloops
@constraint(model, K_petals_in, sum(x[i, 1] for i in V_0) == subloops)
@constraint(model, K_petals_out, sum(x[1, j] for j in V_0) == subloops)

# no subloops that do not connect to home
for S in all_S
    @constraint(model, sum(x[i, j] for i in S, j in S) <= length(S) - 1)
end

# define double_header as: 
# double_header[i, j] == 1 <=> x[home, i] == 1 and x[i, j] == 1 and x[j, home] == 1
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i]))], double_header[i, j] <= x[home, i])
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i]))], double_header[i, j] <= x[i, j])
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i]))], double_header[i, j] <= x[j, home])
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i]))], double_header[i, j] >= x[home, i] + x[i, j] + x[j, home] - 2)

# we want at most DH double-headers
@constraint(model, max_DH_doubleheaders, sum(double_header[i, j] for i in V_0, j in setdiff(V_0, Set([i]))) <= DH)

# define triple_header
# triple_header[i, j] == 1 <=> x[home, i] == 1 and x[i, j] == 1 and x[j, q] == 1 and x[q, home] == 1
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i])), q in setdiff(V_0, Set([i, j]))], triple_header[i, j, q] <= x[home, i])
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i])), q in setdiff(V_0, Set([i, j]))], triple_header[i, j, q] <= x[i, j])
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i])), q in setdiff(V_0, Set([i, j]))], triple_header[i, j, q] <= x[j, q])
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i])), q in setdiff(V_0, Set([i, j]))], triple_header[i, j, q] <= x[q, home])
@constraint(model, [i in V_0, j in setdiff(V_0, Set([i])), q in setdiff(V_0, Set([i, j]))], triple_header[i, j, q] >= x[home, i] + x[i, j] + x[j, q] + x[q, home] - 3)

# we want at most TH triple-headers
@constraint(model, max_TH_tripleheaders, sum(triple_header[i, j, q] for i in V_0, j in setdiff(V_0, Set([i])), q in setdiff(V_0, Set([i, j]))) <= TH)

# we want at most triple-headers, so no more than 3 consecutive races
@constraint(model, max_tripleheaders[i in V_0, j in setdiff(V_0, Set([i])), q in setdiff(V_0, Set([i, j])), r in setdiff(V_0, Set([i, j, q]))], x[home, i] + x[i, j] + x[j, q] + x[q, r] <= 3)

#objective
@objective(model, Min, sum(sum(x[i, j] * distance_matrix[i, j] for i in V) for j in V));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter Threads to value 20


In [90]:
#latex_formulation(model)

In [14]:
optimize!(model)

Set parameter Threads to value 20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 6365 rows, 911 columns and 31584 nonzeros
Model fingerprint: 0xbd45a12c
Variable types: 1 continuous, 910 integer (910 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+02, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 505 rows and 244 columns
Presolve time: 0.08s
Presolved: 5860 rows, 667 columns, 28775 nonzeros
Variable types: 0 continuous, 667 integer (666 binary)
Found heuristic solution: objective 203616.95556

Root relaxation: objective 3.588666e+04, 29 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 35886.6580    0   20 203616.956 35886.6580  82.4%    

In [15]:
objective_value(model)

112036.08607999208

In [16]:
x_opt = value.(x);

In [21]:
df_full = convert_jump_container_to_df(x_opt, dim_names=[:i, :j], value_col=:x)
df = df[df.x .== 1, :]
show(DataFrame([[names(df)]; collect.(eachrow(df))], [:column; Symbol.(axes(df, 1))]), allcols=true, allrows=true)

3×15 DataFrame
 Row │ column  1     2     3     4     5     6     7     8     9     10    11    12    13    14   
     │ String  Real  Real  Real  Real  Real  Real  Real  Real  Real  Real  Real  Real  Real  Real 
─────┼────────────────────────────────────────────────────────────────────────────────────────────
   1 │ i        1     6     1     1     1     8     1     9     5     4     7     2    10     3
   2 │ j        5     4     6     7     2    10     9     8     3     1     1     1     1     1
   3 │ x        1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0